In [1]:
from langchain_ollama import ChatOllama
from llms import get_llama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_chroma import Chroma

from config import Config

print(Config.OLLAMA_MODEL, Config.OLLAMA_CONTEXT_SIZE)

llama3.2 12288


In [2]:
def split_documents(file_path, page_start=None, page_end=None):
    loader = PyMuPDFLoader(file_path=file_path)
    doc = loader.load()[page_start:page_end]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=Config.SPLITTER_CHUNK_SIZE,
        chunk_overlap=Config.SPLITTER_CHUNK_OVERLAP
    )
    docs = text_splitter.split_documents(doc)
    return docs

In [3]:
from pprint import pprint
from langchain_ollama import OllamaEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


def stuff(docs, host, results, key, prompt):
    llm: ChatOllama = get_llama(host=host)

    embeddings_model = OllamaEmbeddings(
        base_url=Config.OLLAMA_EMBEDDINGS_BASE_URL,
        model=Config.OLLAMA_EMBEDDINGS_MODEL
    )
    vectorstore = Chroma(
        collection_name='acordaos',
        embedding_function=embeddings_model,
    )
    vectorstore.add_documents(documents=docs)
    retriever = vectorstore.as_retriever()

    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    print(f"calling {host}")

    result = retrieval_chain.invoke(
        {'input':
            'Comece.'}
    )
    pprint(result)
    # result = format_chain.invoke({'context': result['output_text']})

    print('\a')
    print(f"\nHOST {host}:\n")
    pprint(result)

    # simplificar_prompt = PromptTemplate.from_template(
    #     '''
    #     Simplifique a linguagem usada no texto. Para isso, siga as seguintes diretrizes
    #     '''
    # )

    results[key]['response'] = result['answer']
    results[key]['prompt'] = [
        str(prompt),
    ]

    # history.save('mapreduce', '', result['text'])
    return result

In [4]:
results = {
    "cabecalho": {},
    "relatorio": {},
    "voto": {},
    "decisao": {},
}

In [5]:
files = [
    "documentos/acordaos/0600012-49_REl_28052024_1.pdf",
    "documentos/acordaos/0600059-75_REl_03102023_1.pdf",
    "documentos/acordaos/0600067-56_REl_27102022_1.pdf",
    "documentos/acordaos/0600085-38_RE_25042022_1.pdf",
]

In [6]:
file_path = files[1]

In [7]:
loader = PyMuPDFLoader(file_path=file_path)
doc = loader.load()

In [8]:
prompt = PromptTemplate.from_template("""
Prompt para Geração de Documento Simplificado de Acórdão:

Contexto: Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.

Comando: Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:

### **1. Cabeçalho**

Preencha as seguintes informações:

- Data do Julgamento: Dia em que o tribunal julgou o processo (em texto contínuo).
- Nº DO PROCESSO: Código único para identificar o processo.
- Origem: Juízo e Cidade
- Relator(a): Juiz responsável pelo relatório e voto do caso.
- Recorrente: Parte que entrou com o recurso.
- Recorrido: Parte que defende a decisão anterior.
- Assunto: Indique apenas a questão jurídica central do processo de forma objetiva.

---

### **2. Relatório (O Caso)**

Resuma de forma objetiva os fatos apresentados no acórdão, destacando em 3 parágrafos, com no máximo 3 linhas cada um, e, quando for o caso, caixa de texto explicativa, tudo em texto contínuo.

O QUE DEVE CONTER EM CADA PARÁGRAFO:

2.1. Parágrafo 1: Descreva as informações do processo inicial analisado pelo juiz, indicando:

◦ **O que o autor pediu?**

**◦ O que o réu alegou para se defender?**

2.2. Parágrafo 2: Apresente a decisão do juiz no processo inicial, descrevendo o que o juiz decidiu e as justificativas legais usadas, Cite artigos de lei e fundamentos jurídicos relevantes, considerando:

◦ **O que o juiz decidiu?**

**◦ Quais foram as justificativas legais da decisão?**

**◦ Cite artigos de lei e fundamentos jurídicos relevantes**

2.3. Parágrafo 3: Descreva de forma simples quem recorreu e o que alegou, indicando:

◦ **Quem recorreu?**

**◦ O que alegou para recorrer?**

2.4. Parágrafo 4: Caixa de Texto Explicativa com Termos Jurídicos Relevante para compreensão do assunto principal

◦ **A caixa de texto explicativa tem o objetivo de fornecer definições e explicações simples de termos, expressões ou assuntos jurídicos relevantes para a compreensão do assunto principal do acórdão, ajudando o leitor a entender o contexto e o conteúdo legal envolvido.**

**◦ Exemplo: “Propaganda antecipada negativa: A propaganda eleitoral antecipada negativa acontece quando, antes de 16 de agosto do ano eleitoral (art. 36 da Lei nº 9.504/1997), alguém faz críticas para prejudicar adversários políticos e influenciar eleitores. Essa prática é proibida e pode resultar em multa”.**

---

### 3. Voto (Argumentação/ Motivação/ Fundamentação)

Apresente as razões jurídicas que embasaram a decisão do Tribunal, destacando, em 2 parágrafos, com no máximo 3 linhas, de texto contínuo, considerando:

◦ **Quais foram os princípios legais aplicados.**

**◦ A interpretação dada à legislação.**

**◦ Argumentos relevantes usados na motivação.**

**◦ Use uma linguagem simples e evite jargões técnicos sempre que possível.**

O QUE DEVE CONTER EM CADA PARÁGRAFO:

3.1. Parágrafo 1: Fundamentação de Fato e de Direito: 

◦ **Explique os fatos relevantes.**

**◦ Apresente a fundamentação jurídica, citando artigos de lei, precedentes jurisprudenciais que embasam a decisão do Tribunal.**

3.2. Parágrafo 2: Conclusão e Decisão:

◦ **Conclusão do Voto.**

**◦ Decisão Final com possíveis implicações.**

---

### **4. Resultado do Julgamento (Dispositivo):**

Informe de maneira direta e clara o resultado final da decisão, incluindo em 2 parágrafos, com no máximo 3 linhas em cada um, em texto contínuo:

4.1. Parágrafo 1: Resultado do Julgamento

Aqui, deverá ser informado o resultado final da decisão, se a decisão foi unânime ou não, se manteve a decisão original ou a modificou.

**◦ Resuma o resultado do julgamento de forma clara e objetiva.**

**◦ Exemplo: “O Tribunal Regional Eleitoral do Ceará, por unanimidade, decidiu manter a decisão original, entendendo que...".**

4.2. Parágrafo 2: Decisão Clara e Direta

**◦ Indique de forma concisa apenas uma das opções considerando o PDF ou texto inserido:
- Recurso Negado;
- Recurso Aceito;
- Recurso Parcialmente Aceito.**

**◦ Exemplo: "Recurso Negado."**

---

1. Instruções finais:

5.1. Utilize sempre uma linguagem simples e direta, evitando termos técnicos desnecessários. Caso seja imprescindível usar um termo jurídico específico, certifique-se de explicá-lo claramente.

5.2. Mantenha o número de parágrafos e linhas conforme indicado para cada seção.

5.3. Assegure-se de que as informações jurídicas sejam precisas e que a fundamentação legal esteja diretamente relacionada ao caso em discussão.

5.4. Siga as regras de linguagem simples: seja empático e cordial, use frases curtas, escreva na ordem direta e em voz ativa, coloque as informações mais importantes no início e utilize verbos de ação.

5.5. Revise o texto para garantir que seja claro e acessível, evitando siglas, jargões e termos técnicos. Confirme se todas as diretrizes e restrições foram cumpridas.

5.6. Adote uma escrita humanizada, procurando aproximar o texto da forma como um ser humano se expressaria, para tornar a comunicação mais próxima e compreensível.

### DOCUMENTO

{context}

""")

In [9]:
result = stuff(doc, 1, results, 'cabecalho', prompt)

calling 1
{'answer': '**Cabeçalho**\n'
           '\n'
           'Data do Julgamento: 03 de outubro de 2023\n'
           'Nº DO PROCESSO: 0600059-75\n'
           'Origem: Tribunal Regional Eleitoral do Ceará, Crato/CE\n'
           'Relator(a): Juiz Raimundo Deusdeth Rodrigues Junior\n'
           'Recorrente: Partido dos Trabalhadores - PT\n'
           'Recorrido: Rádio Princesa do Cariri LTDA\n'
           'Assunto: Propaganda eleitoral antecipada negativa\n'
           '\n'
           '**Relatório (O Caso)**\n'
           '\n'
           'O processo em questão envolve uma representação apresentada pelo '
           'Diretório Municipal do Partido dos Trabalhadores no Crato, '
           'alegando que a Rádio Princesa do Cariri LTDA. cometeu propaganda '
           'eleitoral antecipada negativa ao apresentar um programa chamado '
           '"Momento Político" em seu canal de televisão.\n'
           '\n'
           'O programa foi apresentado pelo sensacionalista "âncora" Paulo

In [18]:
pprint(result['answer'])

('Aqui está o documento simplificado de acórdão:\n'
 '\n'
 '**1. Cabeçalho**\n'
 '\n'
 '* Data do Julgamento: 28 de maio de 2024\n'
 '* Nº DO PROCESSO: 0600012-49.2024.6.06.0033\n'
 '* Origem: Tribunal Regional Eleitoral do Ceará, Canindé/CE\n'
 '* Relator(a): Desembargador Eleitoral Daniel Carvalho Carneiro\n'
 '* Recorrente: Antônio Ilomar Vasconcelos Cruz\n'
 '* Recorrido: Promotoria Eleitoral\n'
 '* Assunto: Propaganda eleitoral antecipada\n'
 '\n'
 '**2. Relatório (O Caso)**\n'
 '\n'
 'O caso envolve a propaganda eleitoral antecipada de Antônio Ilomar '
 'Vasconcelos Cruz, candidato a prefeito do município de Canindé. O recorrente '
 'alegou que o candidato realizou propaganda irregular, com pedido explícito '
 'de votos, e que a infraestrutura do evento e o local onde foi realizado '
 'seriam indicativos do objetivo de ampliar o alcance da mensagem de sua '
 'pré-candidatura.\n'
 '\n'
 'O juiz decidiu negar provimento ao recurso, considerando que o ato típico de '
 'campanha elei